# Dwa zbiory danych będą wykorzystane: jeden do uczenia i walidacji, a drugi do testów

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

usecols = ["L3", "F2"]
dataset_train = pd.read_csv('../output/result_triangle_no dist.csv', sep='\t', usecols=usecols, index_col=False)
dataset_test = pd.read_csv('../output/result_steps(whole range)_no dist.csv', sep='\t', usecols=usecols, index_col=False)

dataset_train.head()
dataset_test.head()

plt.figure()
plt.plot(dataset_test['L3'], label='L3')
plt.legend()
plt.figure()
plt.plot(dataset_test['F2'], label='F2')
plt.legend()

plt.figure()
plt.plot(dataset_train['L3'], label='L3')
plt.legend()
plt.figure()
plt.plot(dataset_train['F2'], label='F2')
plt.legend()

plt.figure()
plt.plot(dataset_test['L3'], dataset_test['F2'])
plt.xlabel('L3')
plt.ylabel('F2')
plt.title('Charakterystyka F2(L3)')

# Przygotowanie listy cech i wzorcowych wyjść dla danych uczących i testowych

In [ ]:
train_features = dataset_train.copy()
train_labels = train_features.pop('F2')

test_features = dataset_test.copy()
test_labels = test_features.pop('F2')

# Definicja modelu i uczenie

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Narmalizacja
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

# Warstwa wejść i konstrukcja modelu
input_layer = keras.layers.Input(train_features.shape[1:])
model = keras.Sequential([
      input_layer,
      layers.Dense(2, activation='tanh'),
      layers.Dense(1)
  ])

# Kompilacja modelu
model.compile(loss='mean_absolute_error',
              optimizer=tf.keras.optimizers.Adam(0.02))

# Proce uczenia
history = model.fit(
    train_features,
    train_labels,
    epochs=20,
    validation_split = 0.2)

# Zapis modelu
model.save('../output/F2_L3.keras')

# Sprawdzenie

In [ ]:
# Wczytanie modelu z pliku
model_saved = keras.models.load_model('../output/F2_L3.keras')

# Predykcja
y = model_saved.predict(test_features)

y_ref = test_labels.array
e = [a - b for a, b in zip(y_ref, y)]
plt.figure()
plt.plot(y_ref, label='F2', color='g')
plt.plot(y, label='F3_pred', color='b')
plt.plot(e, label='e', color='r')
plt.legend()

plt.figure()
plt.plot(test_features, test_labels, label='F2(L3)', color='g')
plt.plot(test_features, y, label='F2_pred(L3)', color='b')
plt.legend()
plt.title('Charakterystyka F2(L3)')